In [13]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup

In [14]:
executable_path = {'executable_path': 'c:/util/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Step 1 - Scraping

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.


In [15]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article = soup.find('li', class_='slide')

news_title = article.find('h3').text

news_p = article.find('div', class_='article_teaser_body').text

In [32]:
print(news_title)

Curiosity Says Farewell to Mars' Vera Rubin Ridge


In [33]:
print(news_p)

The Mars rover is moving on and has taken its last selfie on the ridge it's explored for more than a year.
